In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

# DGP for regression

Here we'll show the DGP for regression, using small to medium data sets. 

In [2]:
import numpy as np
import tensorflow as tf
import time

import matplotlib.pyplot as plt
%matplotlib inline 

from gpflow.likelihoods import Gaussian
from gpflow.kernels import RBF, White
from gpflow.mean_functions import Constant
from gpflow.models.sgpr import SGPR, GPRFITC
from gpflow.models.svgp import SVGP
from gpflow.models.gpr import GPR
from gpflow.training import AdamOptimizer, ScipyOptimizer

from scipy.cluster.vq import kmeans2
from scipy.stats import norm
from scipy.special import logsumexp

from doubly_stochastic_dgp.dgp import DGP
from datasets import Datasets
datasets = Datasets(data_path='/data/')

Let's use the kin8nm data set

In [3]:
data = datasets.all_datasets['kin8nm'].get_data()
X, Y, Xs, Ys, Y_std = [data[_] for _ in ['X', 'Y', 'Xs', 'Ys', 'Y_std']]
print('N: {}, D: {}, Ns: {}'.format(X.shape[0], X.shape[1], Xs.shape[0]))

N: 7372, D: 8, Ns: 820


## Single layer models

Our baseline model is a sparse GP, but since the dataset is small we can also train without minibatches so we'll also compare to a collapsed sparse GP (with analytically optimal $q(\mathbf u)$) which is known as SGPR in GPflow terminology, and we'll also cpmpare to FITC

In [4]:
def make_single_layer_models(X, Y, Z):
    D = X.shape[1]
    m_sgpr = SGPR(X, Y, RBF(D), Z.copy())
    m_svgp = SVGP(X, Y, RBF(D), Gaussian(), Z.copy())
    m_fitc = GPRFITC(X, Y, RBF(D), Z.copy())
    for m in m_sgpr, m_svgp, m_fitc:
        m.likelihood.variance = 0.01
    return m_sgpr, m_svgp, m_fitc

Z_100 = kmeans2(X, 100, minit='points')[0]
Z_500 = kmeans2(X, 500, minit='points')[0]
m_sgpr, m_svgp, m_fitc = make_single_layer_models(X, Y, Z_100)
m_sgpr_500, m_svgp_500, m_fitc_500 = make_single_layer_models(X, Y, Z_500)

## DGP models

We'll include a DGP with a single layer here for comparision. We've used a largish minibatch size of $\text{min}(1000, N)$, but it works fine for smaller batches too

In the paper we used 1 sample. Here we'll go up to 10 in celebration of the new implementation (which is much more efficient)

In [5]:
def make_dgp(X, Y, Z, L):
    D = X.shape[1]
    
    # the layer shapes are defined by the kernel dims, so here all hidden layers are D dimensional 
    kernels = []
    for l in range(L):
        kernels.append(RBF(D))
        
    # between layer noise (doesn't actually make much difference but we include it anyway)
    for kernel in kernels[:-1]:
        kernel += White(D, variance=1e-5) 
        
    mb = 1000 if X.shape[0] > 1000 else None 
    model = DGP(X, Y, Z, kernels, Gaussian(), num_samples=10, minibatch_size=mb)

    # start the inner layers almost deterministically 
    for layer in model.layers[:-1]:
        layer.q_sqrt = layer.q_sqrt.value * 1e-5
    
    return model

m_dgp1 = make_dgp(X, Y, Z_100, 1)
m_dgp2 = make_dgp(X, Y, Z_100, 2)
m_dgp3 = make_dgp(X, Y, Z_100, 3)
m_dgp4 = make_dgp(X, Y, Z_100, 4)
m_dgp5 = make_dgp(X, Y, Z_100, 5)

## Prediction

We'll calculate test rmse and likelihood in batches (so the larger datasets don't cause memory problems)

For the DGP models we need to take an average over the samples for the rmse. The `predict_density` function already does this internally


In [6]:
def batch_assess(model, assess_model, X, Y):
    n_batches = max(int(X.shape[0]/1000.), 1)
    lik, sq_diff = [], []
    for X_batch, Y_batch in zip(np.array_split(X, n_batches), np.array_split(Y, n_batches)):
        l, sq = assess_model(model, X_batch, Y_batch)
        lik.append(l)
        sq_diff.append(sq)
    lik = np.concatenate(lik, 0)
    sq_diff = np.array(np.concatenate(sq_diff, 0), dtype=float)
    return np.average(lik), np.average(sq_diff)**0.5

def assess_single_layer(model, X_batch, Y_batch):
    m, v = model.predict_y(X_batch)
    lik = np.sum(norm.logpdf(Y_batch*Y_std, loc=m*Y_std, scale=Y_std*v**0.5),  1)
    sq_diff = Y_std**2*((m - Y_batch)**2)
    return lik, sq_diff 

S = 100
def assess_sampled(model, X_batch, Y_batch):
    m, v = model.predict_y(X_batch, S)
    S_lik = np.sum(norm.logpdf(Y_batch*Y_std, loc=m*Y_std, scale=Y_std*v**0.5), 2)
    lik = logsumexp(S_lik, 0, b=1/float(S))
    
    mean = np.average(m, 0)
    sq_diff = Y_std**2*((mean - Y_batch)**2)
    return lik, sq_diff

## Training 

We'll optimize single layer models and using LFBGS and the dgp models with Adam. It will be interesting to compare the result of `m_svgp` compared to `m_dgp1`: if there is a difference it will be down to the optimizer. 

In [7]:
single_layer_models = [m_sgpr, m_svgp, m_fitc, m_sgpr_500, m_svgp_500, m_fitc_500]
single_layer_names = ['col sgp', 'sgp', 'fitc', 'col sgp 500', 'sgp 500', 'fitc 500']

s = '{:<16}  lik: {:.4f}, rmse: {:.4f}'

for m, name in zip(single_layer_models, single_layer_names):
    ScipyOptimizer().minimize(m, maxiter=5000)
    lik, rmse = batch_assess(m, assess_single_layer, Xs, Ys)
    print(s.format(name, lik, rmse))

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 4570.308663
  Number of iterations: 2250
  Number of functions evaluations: 2321
col sgp           lik: 0.9748, rmse: 0.0866
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 4576.308483
  Number of iterations: 5001
  Number of functions evaluations: 5212
sgp               lik: 0.9749, rmse: 0.0866
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 2080.966339
  Number of iterations: 5001
  Number of functions evaluations: 5298
fitc              lik: 1.1254, rmse: 0.0832
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 2947.244554
  Number of iterations: 5001
  Number of functions evaluations: 5562
col sgp 500 

Now for the DGP models:

In [8]:
for m, name in zip([m_dgp1, m_dgp2, m_dgp3, m_dgp4, m_dgp5], ['dgp1 (sgp+adam)', 'dgp2', 'dgp3', 'dgp4', 'dgp5']):
    AdamOptimizer(0.01).minimize(m, maxiter=5000)
    lik, rmse = batch_assess(m, assess_sampled, Xs, Ys)
    print(s.format(name, lik, rmse))

dgp1 (sgp+adam)   lik: 0.9292, rmse: 0.0913
dgp2              lik: 1.2908, rmse: 0.0664
dgp3              lik: 1.3186, rmse: 0.0647
dgp4              lik: 1.3177, rmse: 0.0649
dgp5              lik: 1.3220, rmse: 0.0645
